In [2]:
# 통신용 모듈
from flask import Flask, jsonify, request
import json

# db용 모듈
import pandas as pd
import cx_Oracle
import numpy as np

# 머신러닝용 모듈


### 사고 순서
>- 식재료 2개 입력 :: ing1,ing2
>- 유저성향 입력 :: userdata=[rain,cold,hot,stress,good,fight,friend,tasty,reward,congratulation]
>- 입력한 식재료가 포함된 요리를 Recipe 테이블에서 출력 :: conn_db(ing1,ing2)
>- conn_db(ing1,ing2) 결과의 TITLE을 가진 food 테이블의 요리 성향 데이터를 출력
>- userdata와 food 테이블에서 출력된 요리 성향 데이터 간의 distance 계산
>- 

In [3]:
# db에 연결, food 테이블 불러오기

# db connection
host="smhrdai.cwnxiom872oa.ap-northeast-2.rds.amazonaws.com"
service="ORCL"
user="smhrd"
pw="hkread132"
query="select * from food"

dsn = cx_Oracle.makedsn(host, 1521, service_name=service)
conn = cx_Oracle.connect(user=user, password=pw, dsn=dsn)

# Get a dataframe
result = pd.read_sql(query, conn)
result = result.set_index("TITLE")

# Close connection
conn.close()

In [4]:
def print_recipe(food_name):
    # db connection
    host="smhrdai.cwnxiom872oa.ap-northeast-2.rds.amazonaws.com"
    service="ORCL"
    user="smhrd"
    pw="hkread132"
    query="select * from recipe where title='"+food_name+"'"

    dsn = cx_Oracle.makedsn(host, 1521, service_name=service)
    conn = cx_Oracle.connect(user=user, password=pw, dsn=dsn)

    # Get a dataframe
    try:
        result = pd.read_sql(query, conn)
        if result.shape[0]==0:
            print('pd.read_sql 결과 없음')
    except:
        result='pd.read_sql에서 오류 발생'
    
    # Close connection
    conn.close()
    return result

In [5]:
# db에 연결: 식재료로 ing1, ing2를 사용하는 레시피 테이블 값 전체 출력
def conn_db(ing1,ing2):
    # db connection
    host="smhrdai.cwnxiom872oa.ap-northeast-2.rds.amazonaws.com"
    service="ORCL"
    user="smhrd"
    pw="hkread132"
    query="select * from recipe where ingredient like '%"+ing1+"%' and ingredient like '%"+ing2+"%'"

    dsn = cx_Oracle.makedsn(host, 1521, service_name=service)
    conn = cx_Oracle.connect(user=user, password=pw, dsn=dsn)

    # Get a dataframe
    try:
        result = pd.read_sql(query, conn)
        if result.shape[0]==0:
            print('pd.read_sql 결과 없음')
    except:
        result='pd.read_sql에서 오류 발생'
    
    # Close connection
    conn.close()
    return result

In [6]:
def dist(new_user, food_name):
    a = np.array(new_user)
    b = np.array(result.loc[food_name, :])

    return np.sqrt(np.sum((a - b)**2))

def main(ing1,ing2, new_user):
    lst = []
    itemset = conn_db(ing1,ing2)
    
    for i in itemset['TITLE']:
        lst.append(dist(new_user, i))
    
    lst = pd.Series(lst)
    lst.index =itemset["TITLE"]
    lst = lst.sort_values()
    return lst[:1].index

In [ ]:
app = Flask(__name__)

# variable initialization: the host address and port adress
host_addr="220.95.45.219"
port_addr=9010
# it doesn't work when write in app.run() directly
# what is fucking "HELLO FLASH"?

# connect to http://host_addr:port_addr/routing_address
# http://localhost:9010/sendValuetest
@app.route("/SendSurveyResult", methods=['POST']) # app.route("/routing_address")
def SendSurveyResult(): # define a function when the request is in this URL
    print('connect')
#     data=request.get_json()
#     data=request.json
    data=request.args
    print('넘어온 값={}'.format(data))
    if request.method=='POST':
        print('method=POST')
        try:
#             ing1=data['ing1']
            ing1=request.form.get('ing1')
        except:
            ing1='NO'
        
        try:
#             ing2=data['ing2']
            ing2=request.form.get('ing2')
        except:
            ing2='NO'
            
        print('ing1={}'.format(ing1))
        print('ing2={}'.format(ing2))

        # Android로부터 받아온 파라미터값(FoodDTO)
#         rain=int(data['rain'])
        rain=int(request.form.get('rain'))
        print('rain={}'.format(rain))
        
#         cold=int(data['cold'])
        cold=int(request.form.get('cold'))
        print('cold={}'.format(cold))
        
#         hot=int(data['hot'])
        hot=int(request.form.get('hot'))
        print('hot={}'.format(hot))
        
#         stress=int(data['stress'])
        stress=int(request.form.get('stress'))
        print('stress={}'.format(stress))
        
#         good=int(data['good'])
        good=int(request.form.get('good'))
        print('good={}'.format(good))
        
#         fight=int(data['fight'])
        fight=int(request.form.get('fight'))
        print('fight={}'.format(fight))
        
#         friend=int(data['friend'])
        friend=int(request.form.get('friend'))
        print('friend={}'.format(friend))
        
#         tasty=int(data['tasty'])
        tasty=int(request.form.get('tasty'))
        print('tasty={}'.format(tasty))
        
#         reward=int(data['reward'])
        reward=int(request.form.get('reward'))
        print('reward={}'.format(reward))
        
#         congratulation=int(data['congratulation'])
        congratulation=int(request.form.get('congratulation'))
        print('congratulation={}'.format(congratulation))

        
        s=main(ing1,ing2,[rain,cold,hot,stress,good,fight,friend,tasty,reward,congratulation])
        print('추천 요리='+s)
        
        return_s=print_recipe(s[0])
        
        return_s=return_s.to_json()
        
    return return_s

if __name__ == "__main__": # where is the field "__main__"?
    app.run(host=host_addr, port=port_addr)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://220.95.45.219:9010/ (Press CTRL+C to quit)
